In [20]:
import pandas as pd
import os
import numpy as np

def interpolate_df(df):
    df = df.sort_values('frame')
    df['x']=df.x.interpolate()
    df['y']=df.y.interpolate()
    return df

def smooth(d, var, winmed, winmean):
    winmed1 = winmed
    winmed = int(winmed*d.fps.unique()[0])
    winmean = int(winmean*d.fps.unique()[0])
    d = d.reset_index(drop=True)
    if winmed>0:
        x = d.sort_values('frame')[var].rolling(center=True,window=winmed).median()
        d[var] = x.rolling(center=True,window=winmean).mean()
    else:
        d[var] = d[var]
    d['smooth'] = winmed1
    return d

def comp_joint_angle(df, joint_str):
    df = df.loc[(df.bp=='L'+joint_str)|(df.bp=='R'+joint_str)]
    df = pd.pivot_table(df, columns = ['bp'], values=['x', 'y'], index=['frame'])
    # zangle =np.arctan2(Rj.y.iloc[0]-Lj.y.iloc[0],Rj.x.iloc[0]-Lj.x.iloc[0])
    df[joint_str+'_angle']= np.arctan2((df['y', 'R'+joint_str]-df['y', 'L'+joint_str]),(df['x', 'R'+joint_str]-df['x', 'L'+joint_str]))
    df = df.drop(['x', 'y'], axis=1)
    return df

def comp_center_joints(df, joint_str, jstr):
    df = df.loc[(df.bp=='L'+joint_str)|(df.bp=='R'+joint_str)]
    df = pd.pivot_table(df, columns = ['bp'], values=['x', 'y'], index=['frame'])
    # zangle =np.arctan2(Rj.y.iloc[0]-Lj.y.iloc[0],Rj.x.iloc[0]-Lj.x.iloc[0])
    df[jstr+'y']= (df['y', 'R'+joint_str]+df['y', 'L'+joint_str])/2
    df[jstr+'x']= (df['x', 'R'+joint_str]+df['x', 'L'+joint_str])/2
    df = df.drop(['x', 'y'], axis=1)
    return df

def normalise_skeletons(df):
    
    ''' Rotate keypoints around reference points (center of shoulders, center of hips)\
    Normalise points by reference distance (trunk length)'''

    bps = ["Nose","Neck","RShoulder","RElbow","RWrist","LShoulder","LElbow","LWrist","RHip","RKnee","RAnkle","LHip","LKnee",\
     "LAnkle","REye","LEye","REar","LEar"]
    ubps = ["Nose","Neck","RShoulder","RElbow","RWrist","LShoulder","LElbow","LWrist", "REye","LEye","REar","LEar"]
    lbps = ["RKnee","RAnkle","LHip","LKnee","LAnkle","RHip"]
    u_idx = np.where(np.isin(bps, ubps)==1)[0]
    l_idx = np.where(np.isin(bps, lbps)==1)[0]
    df['upper'] = np.isin(df.bp, ubps)*1
    # compute shoulder and hip angles for rotating, upper and lower body 
    # reference parts, now center of shoulders and hips
    
    s_angle = df.groupby(['video']).apply(lambda x: comp_joint_angle(x,'Shoulder')).reset_index()
    h_angle = df.groupby(['video']).apply(lambda x: comp_joint_angle(x,'Hip')).reset_index()
    uref = df.groupby(['video']).apply(lambda x: comp_center_joints(x, 'Shoulder', 'uref')).reset_index()
    lref = df.groupby(['video']).apply(lambda x: comp_center_joints(x, 'Hip', 'lref')).reset_index()
    s_angle['Hip_angle'] = h_angle['Hip_angle']
    s_angle = pd.merge(s_angle, uref, on=['video', 'frame'], how='inner')
    s_angle = pd.merge(s_angle, lref, on=['video', 'frame'], how='inner')
    s_angle.columns = s_angle.columns.get_level_values(0)

    df = pd.merge(df,s_angle, on=['video', 'frame'], how = 'outer')
    # set up columns, reference parts and reference angles

    df['refx'] = df['urefx']*df['upper'] + df['lrefx']*(1-df['upper'])
    df['refy'] = df['urefy']*df['upper'] + df['lrefy']*(1-df['upper'])
    df['ref_dist'] = np.sqrt((df['urefy']-df['lrefy'])**2+(df['urefx']-df['lrefx'])**2)
    df['ref_angle'] = df['Shoulder_angle']*df['upper'] + df['Hip_angle']*(1-df['upper'])

    df.loc[df.ref_angle<0,'ref_angle'] = 2*np.pi + df.loc[df.ref_angle<0,'ref_angle'] 
    df.loc[df.ref_angle<np.pi,'ref_angle'] = np.pi - df.loc[df.ref_angle<np.pi,'ref_angle'] 
    df.loc[(df.ref_angle>np.pi)&(df.ref_angle<2*np.pi),'ref_angle'] = 3*np.pi - df.loc[(df.ref_angle>np.pi)&(df.ref_angle<2*np.pi),'ref_angle']
    df['x_rotate'] = df['refx'] + np.cos(df['ref_angle'])*(df['x']-df['refx']) - np.sin(df['ref_angle'])*(df['y'] - df['refy'])
    df['y_rotate'] = df['refy'] + np.sin(df['ref_angle'])*(df['x']-df['refx']) + np.cos(df['ref_angle'])*(df['y'] - df['refy'])
    df['x_rotate'] = (df['x_rotate']-df['refx'])/df['ref_dist']
    df['y_rotate'] = (df['y_rotate']-df['refy'])/df['ref_dist']
    df['x'] = df['x_rotate']
    df['y'] = df['y_rotate']
    # add to lower body to make trunk length 1
    df.loc[df.upper==0,'y'] = df.loc[df.upper==0,'y']+1
    df['delta_t'] = 1/df['fps']
    return df

def get_joint_angles(df):
    
    ''' Compute joint angles from x,y coordinates '''

    df = df[~df.x.isnull()]
    df['delta_t'] = 1/df['fps']

    bps = ["Nose","Neck","RShoulder","RElbow","RWrist","LShoulder","LElbow","LWrist","RHip","RKnee","RAnkle","LHip","LKnee",\
     "LAnkle","REye","LEye","REar","LEar"]
    joints = [['part0', 'part1', 'part2'],\
    ['RShoulder','Neck', 'RElbow'],\
    ['RElbow', 'RShoulder','RWrist'],\
    ['RHip','LHip', 'RKnee'],\
    ['RKnee', 'RHip','RAnkle'],\
    ['LShoulder','Neck', 'LElbow'],\
    ['LElbow', 'LShoulder','LWrist'],\
    ['LHip','RHip', 'LKnee'],\
    ['LKnee', 'LHip','LAnkle']]
    headers = joints.pop(0)
    df_joints = pd.DataFrame(joints, columns=headers).reset_index()
    df_joints['bpindex'] = df_joints['index']+1

    df_1 = df
    for i in [0,1,2]:
        df_joints['bp'] = df_joints['part'+str(i)]
        df_1 = pd.merge(df_1,df_joints[['bp', 'bpindex']], on='bp', how='left')
        df_1['idx'+str(i)] = df_1['bpindex'] 
        df_1 = df_1.drop('bpindex', axis=1)
        df_1['x'+str(i)] = df_1['x']*df_1['idx'+str(i)]/df_1['idx'+str(i)]
        df_1['y'+str(i)] = df_1['y']*df_1['idx'+str(i)]/df_1['idx'+str(i)]
    df0 = df_1[['video', 'frame', 'idx0', 'x0', 'y0', 'bp']]; df0 = df0.rename(index=str, columns={"bp": "bp0", "idx0": "idx"}); df0 = df0[~df0.idx.isnull()]
    df1 = df_1[['video', 'frame', 'idx1', 'x1', 'y1', 'bp']]; df1 = df1.rename(index=str, columns={"bp": "bp1", "idx1": "idx"}); df1 = df1[~df1.idx.isnull()]
    df2 = df_1[['video', 'frame', 'idx2', 'x2', 'y2', 'bp']]; df2 = df2.rename(index=str, columns={"bp": "bp2", "idx2": "idx"}); df2 = df2[~df2.idx.isnull()]
    df_2 = pd.merge(df0,df1, on=['video', 'frame', 'idx'], how='inner')
    df_2 = pd.merge(df_2,df2, on=['video', 'frame', 'idx'], how='inner')

    # compute angle here
    df_2['dot'] = (df_2['x1'] - df_2['x0'])*(df_2['x2'] - df_2['x0']) + (df_2['y1'] - df_2['y0'])*(df_2['y2'] - df_2['y0'])
    df_2['det'] = (df_2['x1'] - df_2['x0'])*(df_2['y2'] - df_2['y0']) - (df_2['y1'] - df_2['y0'])*(df_2['x2'] - df_2['x0'])
    df_2['angle_degs'] = np.arctan2(df_2['det'],df_2['dot'])*180/np.pi
    # hip and shoulder should be same regardless of side
    # elbow/knee give flexion/extension information only
    df_2['side'] = df_2.bp0.str[:1]
    df_2['part'] = df_2.bp0.str[1:]
    df_2['angle'] = df_2.angle_degs # same on left/right
    df_2.loc[(df_2['bp0']=='LShoulder')|(df_2['bp0']=='LHip'),'angle'] = \
    df_2.loc[(df_2['bp0']=='LShoulder')|(df_2['bp0']=='LHip'),'angle']*(-1)
    df_2.loc[(df_2['part']=='Elbow')|(df_2['part']=='Knee'),'angle'] = \
    np.abs(df_2.loc[(df_2['part']=='Elbow')|(df_2['part']=='Knee'),'angle'])
    # shoulders/hips: change to -180-+180 to 0-360 if neg: 360+angle
    df_2.loc[((df_2['part']=='Shoulder')|(df_2['part']=='Hip'))& (df_2.angle<0),'angle'] = \
    df_2.loc[((df_2['part']=='Shoulder')|(df_2['part']=='Hip'))& (df_2.angle<0),'angle']+360
    # can include shoulder rotation
    df_2['bp'] = df_2['bp0']

    df_info = df.groupby(['video', 'frame', 'fps','time', 'delta_t']).mean().reset_index()[['video', 'frame', 'fps','time', 'delta_t']]
    df_angle = pd.merge(df_2[['video', 'frame', 'bp', 'side', 'part', 'angle']],\
    df_info, on=['video', 'frame'], how='inner').drop_duplicates()
    return df_angle


In [21]:
data_set = 'youtube'
pose_estimates_path = '../data/pose_estimates/'+data_set+'/py'

median_window = 1
mean_window = 1

print('load')
df = pd.read_pickle(os.path.join(pose_estimates_path, 'pose_estimates.pkl'))

print('x_vid')
# normalise x and y by image length (conserve aspect ratio)
df['x'] = pd.to_numeric(df['x'])
df['y'] = pd.to_numeric(df['y'])
df['x'] = (df['x'] - df['pixel_x']/2)/df['pixel_y']
df['y'] = (df['y'] - df['pixel_y']/2)/df['pixel_y']

print('interpolate')
# interpolate
df = df.groupby(['video', 'bp']).apply(interpolate_df).reset_index(drop=True)

print('smooth')
# median and mean filter
median_window = .5
mean_window = .5
df = df.groupby(['video', 'bp']).apply(lambda x: smooth(x, 'y', median_window, mean_window)).reset_index(drop=True)
df = df.groupby(['video', 'bp']).apply(lambda x: smooth(x, 'x', median_window, mean_window)).reset_index(drop=True)

# remove nans added by filtering
# df = df.loc[~df.x.isnull(),:]

print('normalise')
# rotate and normalise by reference
df = normalise_skeletons(df)

print('getting joint angles')
# extract angles
df_angle = get_joint_angles(df)

# save
df.to_pickle(os.path.join(pose_estimates_path, 'processed_pose_estimates_coords.pkl'))
df_angle.to_pickle(os.path.join(pose_estimates_path, 'processed_pose_estimates_angles.pkl'))

load
x_vid
interpolate
smooth
normalise


/usr/local/lib/python3.5/dist-packages/pandas/core/generic.py:2530: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


getting joint angles


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [19]:
# get pose animations
# save animation with right fps

# order

# load_pose_data
# preprocess_pose_data
# viz_pose_data

# build_features
# merge_data_sets
# compute_surprise
# generate_plots
